In [7]:
from langchain.llms import OpenAI
import os

In [9]:
openai_api_key = os.getenv(key="OPENAI_API_KEY")
llm = OpenAI(openai_api_key=openai_api_key)

## Language Model Templates

In [10]:
planet = "Saturn"

llm(f"Here is a fact about {planet}:")

' Saturn is the sixth planet from the Sun and the second largest planet in our solar system. It is known for its iconic rings made of ice and rock particles, which can be seen from Earth through a telescope. Saturn has at least 82 moons, with the largest being Titan. The planet has a thick atmosphere primarily made of hydrogen and helium, and it has the fastest winds in the solar system, reaching speeds of up to 1,100 miles per hour. Saturn has been explored by multiple spacecraft, including the Cassini-Huygens mission which arrived in 2004 and provided detailed images and data about the planet and its moons.'

In [10]:
from langchain import PromptTemplate

no_input_prompt = PromptTemplate(input_variables=[], template="Tell me a fact")
no_input_prompt.format()

'Tell me a fact'

In [12]:
llm(no_input_prompt.format())

'\n\nThe shortest war in history lasted only 38 minutes. It was between Great Britain and Zanzibar on August 27, 1896.'

In [11]:
single_input_prompt = PromptTemplate(
    input_variables=["topic"], template="Tell me a fact about {topic}."
)

In [12]:
llm(single_input_prompt.format(topic="the bottom of the ocean"))

"\n\nThe bottom of the ocean, also known as the seafloor, covers approximately 70% of the Earth's surface and contains the longest mountain range in the world, known as the Mid-Ocean Ridge."

In [14]:
multi_input_prompt = PromptTemplate(
    input_variables=["topic", "level"],
    template="Tell me a fact about {topic} for a {level} student.",
)

In [17]:
llm(multi_input_prompt.format(topic="the bottom of the ocean", level="2nd grade"))

'\n\nThe bottom of the ocean is called the "seafloor" and it is covered in sand, rocks, and plants just like the land on Earth.'

# Chat Model Templates

Chat models require prompts consisting of chat messages with assigned roles such as AI, human, or system. 

The OpenAI Chat Completion API assigns roles to messages, with the model focusing more on instructions from system messages.

LangChain provides prompt templates for easier prompt construction, recommended over PromptTemplate for better interaction with chat models.

In [18]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(openai_api_key=openai_api_key)

In [31]:
system_template = "You are an AI recipe assistant that specializes in {dietary_preference} dishes that can be prepared in {cooking_time}."
human_template = "{recipe_request}"

system_message_prompt = SystemMessagePromptTemplate.from_template(
    template=system_template
)
human_message_prompt = HumanMessagePromptTemplate.from_template(template=human_template)

The human message is what the user asks the assistant

In [20]:
print(system_message_prompt.input_variables)
print(human_message_prompt.input_variables)

['cooking_time', 'dietary_preference']
['recipe_request']


In [21]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [22]:
chat_prompt.input_variables

['cooking_time', 'dietary_preference', 'recipe_request']

In [28]:
prompt = chat_prompt.format_prompt(
    cooking_time="30 min", dietary_preference="Italian", recipe_request="Quick Snack"
).to_messages()

result = chat(prompt)

In [29]:
print(result.content)

Italian Bruschetta

Ingredients:
- 1 baguette or Italian bread
- 4 ripe tomatoes
- 2 cloves of garlic
- Fresh basil leaves
- Extra virgin olive oil
- Salt and pepper to taste

Instructions:
1. Preheat your oven to 375°F (190°C).
2. Slice the baguette or Italian bread into about 1-inch thick slices.
3. Place the bread slices on a baking sheet and drizzle with olive oil.
4. Toast the bread slices in the oven for about 5-7 minutes, or until they are lightly golden and crispy.
5. While the bread is toasting, dice the tomatoes and finely chop the garlic cloves.
6. In a bowl, combine the diced tomatoes, chopped garlic, a handful of torn fresh basil leaves, a drizzle of olive oil, and salt and pepper to taste. Mix well.
7. Remove the toasted bread slices from the oven and let them cool slightly.
8. Once the bread slices have cooled, spoon the tomato mixture onto each slice.
9. Serve immediately and enjoy your quick and delicious Italian bruschetta!


## Let's wrap it all in a function

We want to build a function that receives an hobby and a budget as parameters and outputs some ideas related to the hobby whithin the specified budget.

In [8]:
def hobby_idea(interest,budget):
    '''
    Inputs:
        interest: A str interest or hobby (e.g. fishing)
        budget: A str budget (e.g. $1000)
    '''
    system_template: str = "You are an helpful assistant that is an expert in {interest}."
    human_template: str = "Suggest me some ideas with a budget of {budget}"

    system_message_prompt = SystemMessagePromptTemplate.from_template(template = system_template)
    human_message_prompt = HumanMessagePromptTemplate.from_template(template = human_template)

    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

    prompt = chat_prompt.format_prompt(interest = interest, budget = budget).to_messages()

    result = chat(prompt)
    
    return result.content

In [17]:
print(hobby_idea('board games','$200'))

Sure! Here are some board game ideas within a budget of $200:

1. Settlers of Catan: This classic strategy game is perfect for 3-4 players and offers endless replayability with its modular board and trading mechanics.

2. Ticket to Ride: A popular gateway game, Ticket to Ride challenges players to build train routes across North America, Europe, or other locations. It's easy to learn but still offers strategic depth.

3. Terraforming Mars: In this highly immersive game, players compete to terraform the planet Mars by strategically managing resources, building infrastructure, and completing projects.

4. Scythe: Set in an alternate 1920s Europe, Scythe combines elements of resource management, area control, and strategic combat. The stunning artwork and high-quality components make it a visually impressive game.

5. Gloomhaven: If you're looking for a cooperative adventure game with a deep and immersive storyline, Gloomhaven is a great choice. Players take on the roles of mercenaries ex

Let's build now a simple AI advisor to help classify incoming customer contacts.

In [15]:
def AI_advisor(advisor_function,ticket):
    '''
    Inputs:
        advisor_function: Function of the advisor (e.g: Customer Service)
        ticket: Customer contact (e.g: my order camed damaged, I want a refund.)
    '''
    system_template: str = "You are an helpful assistant that is an expert in {advisor_function}."
    human_template: str = "What is the most suitable category for the following customer contact: {ticket}"

    system_message_prompt = SystemMessagePromptTemplate.from_template(template = system_template)
    human_message_prompt = HumanMessagePromptTemplate.from_template(template = human_template)

    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

    prompt = chat_prompt.format_prompt(advisor_function = advisor_function, ticket = ticket).to_messages()

    result = chat(prompt)
    
    return result.content

In [16]:
AI_advisor(advisor_function="Customer Service", ticket="My order is delayed, I want a full refund and to cancel the order.")

'The most suitable category for this customer contact would be "Order inquiries" or "Order management". This category typically handles issues related to order delays, cancellations, and refunds.'